In [2]:
from langchain.chains.router import MultiPromptChain # chain for routing between multiple prompts
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser # use LLM to route between subchains
from langchain.prompts import PromptTemplate



## Router Chain 

Route an input to a chain , based on what is input

Multiple sub-chains ,  routerchain decides which chain to pass input to.

In [85]:
# example using subjects
# define String templates to be used later
physics_template = """You are a Physics Professor. You need to answers questions about physics in a concise and 
easy to understand manner. When you do now know the answer to a question , you will admit it.
Here is the question : 
{input}
"""

maths_template = """You are the best Mathematician ever. You are great at answering and solving Math questions.
you are so good that you can break up hard problems and into components , solve these components independently, 
and then put them together to answer broader question.
Here's the question/problem:
{input}
"""

history_template = """
You are a historian. You have excellent knowledge and understanding of people , events and contexts from a range
of historical periods. YOu have the ability to think , reflect , debate , discuss and  evaluate the past. You have
the ability to reason and answer historical questions.
Here's the question:
{input}
"""


cs_template = """ 
You are a prefessional Computer Scientist. You have strong problem solving skills and excellent knowledge of 
computer-sceince concepts. You are able to answer coding questions and able to provide imperative steps to 
implement the solution. You know the tradeoffs between different solutions and are able to select the best solution
based on question's requirement.
Here's the question:
{input}
"""

# here use of {input} is important , for chaining and ouput parsing

In [86]:
# define a structure for more information about each prompt.
# this structure is passed to router-chain , so router chain can decide which sub-chain to use
prompt_infos = [
    {
        "name" : "physics",
        "description" : "prompt for answering physics question",
        "prompt_template" : physics_template
    },
    {
        "name": "maths",
        "description" : "prompt for answering mathematics related question",
        "prompt_template" : maths_template
    },
    {
        "name": "history",
        "description":"prompt for answering history related questions",
        "prompt_template" : history_template
    },
    {
        "name" : "computer science",
        "description" : "prompt for answering computer science questions",
        "prompt_template" : cs_template
    }
]

- MultipromptChain - used when routing between different types of prompt templates
- LLMRouterChain - use language model itself to route between different subchains , the `description` and `name`
  provided above will be used here
- RouterParser - LLM output into a dictionary , which can be used later to select input for subchains


In [87]:


from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv,find_dotenv

_ = load_dotenv(find_dotenv())
chat_model = ChatOpenAI(temperature=0.0)

In [75]:
# create destination chains 
# this will be used by LLMRouterChain

destination_chains = {} # map name of subject to a chain
for prompt_info in prompt_infos:
    name = prompt_info['name']
    prompt_template = prompt_info['prompt_template']
    prompt = PromptTemplate.from_template(template = prompt_template)
    chain = LLMChain(llm = chat_model , prompt= prompt )
    destination_chains[name] = chain

# import pprint
# pprint.pprint(destination_chains)

destinations = [f"{p['name']} : {p['description']}" for p in prompt_infos] # list of string -> `subject : description`

destination_str = "\n".join(destinations)

In [88]:
# create a default prompt , when a question of different subject is asked
from langchain.prompts import ChatPromptTemplate
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat_model , prompt=default_prompt)

In [2]:
# define a template to be used by LLM to route between different chains
# the formatting and words used in this template are important and used by router chain to generate intermeditary ouput
MULTI_PROMPT_ROUTER_TEMPLATE = """
Given a raw text input to a language model, select the select the model prompt best suited for input.
You will be given names of prompt and a description of what the prompt can do. You may also revise the input
if it leads to better response from language model.

<<FORMATTING>>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination" : string - name of the prompt to use or "DEFAULT"
    "next_inputs" : string - a potentially modified version of original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below or it can be "DEFAULT" 
if the input is not well suited for any of the candidate prompts. Do not add or generate "destination" on your own.

REMEMBER: "next_inputs" can be same as original input if you do not think any modification is needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >> 
{{input}}

<< OUTPUT (remember to include the ```json)>> 
""" # quadruple and double {} are for placeholders in multiple line f-strings
# destination , next_inputs are expected by RouterOuputParserClass
#print(MULTI_PROMPT_ROUTER_TEMPLATE)


Given a raw text input to a language model, select the select the model prompt best suited for input.
You will be given names of prompt and a description of what the prompt can do. You may also revise the input
if it leads to better response from language model.

<<FORMATTING>>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination" : string - name of the prompt to use or "DEFAULT"
    "next_inputs" : string - a potentially modified version of original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below or it can be "DEFAULT" 
if the input is not well suited for any of the candidate prompts. Do not add or generate "destination" on your own.

REMEMBER: "next_inputs" can be same as original input if you do not think any modification is needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >> 
{{input}}

<< OUTPUT (remember to include the ```json)>> 



In [136]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations = destination_str
)
router_prompt = PromptTemplate(
    template = router_template,
    input_variables = ["input"],
    output_parser = RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(llm= chat_model , prompt=router_prompt)

In [137]:
chain = MultiPromptChain(
    router_chain= router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True
)

In [138]:
chain.run("What is subconcious ?")



> Entering new MultiPromptChain chain...
None: {'input': 'What is subconcious ?'}
> Finished chain.


'The subconscious mind refers to the part of our mind that operates below the level of conscious awareness. It is responsible for storing and processing information, beliefs, memories, and emotions that are not currently in our conscious awareness. The subconscious mind influences our thoughts, feelings, and behaviors, often without us being aware of it. It is believed to play a significant role in shaping our personality, habits, and responses to various situations.'